In [62]:
# dependencies
using Polynomials

# First Implementation 
Direction: 
- Evaluate the function at various points around the edges of the rectangle. 
- Count how many cycles the argument completes while traversing around the rectangle.

Spot rectangle, and how many zeros are inside.
By tracking the winding number, we can locate the number of zeros in the holomorphic function

Wind(arg) = 1/(2*pi*im) * integrate(1/(x+im*y))

In [18]:
# define data structure of rectangle
struct Rec
    btmlft::Any
    uplft::Any
    uprght::Any
    btmrght::Any
end

# need or not?
# struct Step
#     height::Float32
#     width::Float32
# end

In [64]:
# calculate height and width
function parseInput(input)
    height = input.uplft[2] - input.btmlft[2]
    width = input.btmrght[1] - input.btmlft[1]
end

# change n accordingly so argument analysis is comprehensive
n = 50
step = height/n

0.2

In [3]:
# evaluate the argument at various points starting at bottomleft and traversing one round
# store arguments in array

function argBox(f, input)
    # bottom left to top left
    i = input.btmlft[2]
    argArray = []
    while i < input.uplft[2]
        arg = atan(i/input.btmlft[1])  # this is where the input function comes in for evaluation
        push!(argArray,arg)
        i += step
    end
    
    # top left to top right
    j = input.uplft[1]
    while j < input.uprght[1]
        arg = atan(input.uplft[2]/j)
        push!(argArray,arg)
        j += step
    end
    
    # top right to bottom right
    i = input.uprght[2]
    while i > input.btmrght[2]
        arg = atan(i/input.uprght[1])
        push!(argArray,arg)
        i -= step
    end
    
    # bottom right to bottom left
    j = input.btmrght[1]
    while j > input.btmlft[1]
        arg = atan(input.btmlft[2]/j)
        push!(argArray,arg)
        j -= step
    end
    
    return argArray
end


argBox (generic function with 1 method)

In [2]:
# spot jumps (~>= 2pi or 6)
# the output of argBox(input) is an array of the arguments
# use a for loop to check if any of the pairs match the criteria
# increase count and return the final count. This is the number of zeros in the locus


function countJump(f, input)
    arr = argBox(f, input)
    len = length(arr)
    count = 0
    
    for i in 1:(len - 1)
        if abs(arr[i] - arr[i+1]) > 6    # decrement if it goes clockwise (less than -6)
            count += 1
        end
    end
    return count
end

countJump (generic function with 1 method)

## Unit Tests
Test the algorithm properly. Polynomials are easy to put the zeros exactly where you want them. Try multiplying another function which has no zero in the region to make it more computationally expensive.

In [65]:
# first unit test with 4 zeros
t1 = Polynomial([-1+2im,1+4im,-3im, 4+6im], :x)
input = Rec((1,1), (1,10), (10,10), (10,1))
parseInput(input)
countJump(t1, input) = 2

countJump (generic function with 1 method)